In [7]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

!rm -rf ./logs/GRU

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [13]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename):
    with open(filename, 'rb') as f:
        data_x = np.load(f)
        data_y = np.load(f)
        
    data_x = convert_3D(data_x)
    return data_x, data_y

In [14]:
training_data_x, training_data_y = features_and_labels("../archive/clean-exo-train.npy")
test_x, test_y = features_and_labels("../archive/clean-exo-test.npy")

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

(5087, 3197, 1)
(570, 3197, 1)


In [5]:
GRUmodel = Sequential([
    GRU(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    GRU(64),
    #Dropout(0.3),
    #GRU(64),
    Dense(1, activation='sigmoid')
])

GRUmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy"]
)

log_dir = "logs/GRU/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

csv_logger = CSVLogger('keras-models/KerasGRU/training.csv',separator=",", append=False)

GRUmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = GRUmodel
model.save('keras-models/KerasGRU')

Epoch 1/10
112/112 [==============================] - 28s 203ms/step - loss: 0.6684 - accuracy: 0.9919 - val_loss: 0.6485 - val_accuracy: 0.9935
Epoch 2/10
112/112 [==============================] - 20s 178ms/step - loss: 0.6288 - accuracy: 0.9924 - val_loss: 0.6075 - val_accuracy: 0.9935
Epoch 3/10
112/112 [==============================] - 20s 178ms/step - loss: 0.5855 - accuracy: 0.9924 - val_loss: 0.5612 - val_accuracy: 0.9935
Epoch 4/10
112/112 [==============================] - 20s 179ms/step - loss: 0.5356 - accuracy: 0.9924 - val_loss: 0.5067 - val_accuracy: 0.9935
Epoch 5/10
112/112 [==============================] - 20s 179ms/step - loss: 0.4758 - accuracy: 0.9924 - val_loss: 0.4409 - val_accuracy: 0.9935
Epoch 6/10
112/112 [==============================] - 20s 180ms/step - loss: 0.4042 - accuracy: 0.9924 - val_loss: 0.3629 - val_accuracy: 0.9935
Epoch 7/10
112/112 [==============================] - 20s 179ms/step - loss: 0.3211 - accuracy: 0.9924 - val_loss: 0.2751 - val_ac

INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


In [6]:
%tensorboard --logdir logs/GRU/fit